In [20]:
import numpy as np
import torch
from alogcv.diagonal import diagonal

import linops as lo
# commit: 07a76f6

In [34]:
torch.manual_seed(0)
n0 = 100
p0 = 80
scale = 10

def generate_data(n, p, k_sparse=None):

    if k_sparse is None:
        k_sparse = p

    X = torch.randn(n, p)

    beta = torch.zeros(p)
    beta[:k_sparse] = torch.randn(k_sparse) / np.sqrt(k_sparse)

    y = X @ beta + torch.randn(n)
    return X, y, beta

n = n0 * scale
p = p0 * scale
X, y, beta = generate_data(n, p)
lamda = 0.1

y.requires_grad = True
H = X @ torch.linalg.solve(X.T @ X + n * lamda * torch.eye(p), X.T)
y_hat = H @ y
diagonal(y_hat, y, 'xdiag', {'m': 10})

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn